In [1]:
import pymongo

In [2]:
import pandas as pd

In [3]:
from core.config import MONGODB_URL, DATABASE_NAME, Feedback_Label_Collection, LABEL_COLLECTION

In [5]:
col = client[DATABASE_NAME][Feedback_Label_Collection]

/home/eason/anaconda3/envs/adapter/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


47960

In [4]:
client = pymongo.MongoClient(MONGODB_URL)

col = client[DATABASE_NAME][Feedback_Label_Collection]

result = col.find({"user": "CONLL_0818@admin"})

result = list(result)

from tqdm import tqdm
df = pd.DataFrame()
for i, sentence in enumerate(tqdm(result)):
    sentense_df = pd.DataFrame(columns=["Sentence #", "text", "labels"], data = sentence["text_and_labels"])
    sentense_df["Sentence #"] = str(sentence["_id"])
    df = df.append(sentense_df)

df["multiTags"] = df.labels.apply(lambda x: "|".join(x))

 19%|█████▊                         | 9068/47959 [00:53<03:50, 168.88it/s]


KeyboardInterrupt: 

In [ ]:
result = list(result)

In [ ]:
label_name = "Party87"



In [ ]:
train_data_search_filter = {}

In [ ]:
from tqdm import tqdm
def get_training_dataframe(train_data_search_filter = {}):

    col = client[DATABASE_NAME][Feedback_Label_Collection]

    result = col.find(train_data_search_filter)

    df = pd.DataFrame()
    for i, sentence in enumerate(tqdm(result)):
        sentense_df = pd.DataFrame(columns=["Sentence #", "text", "labels"], data = sentence["text_and_labels"])
        sentense_df["Sentence #"] = str(sentence["_id"])
        df = df.append(sentense_df)
    return df
def get_training_data_by_df_according_to_label_name(df, label_name):
    label_col = client[DATABASE_NAME][LABEL_COLLECTION]

    label_info = label_col.find_one({"label_name": label_name})
    alias_labels = label_col.find({"alias_as": {"$in": [label_name]}})

    alias = []
    for alias_label in alias_labels:
        alias.append(alias_label["label_name"])
    alias = alias + label_info["inherit"]

    wanted_label = [label_name] + alias

    def label_data(label):
        if set(label).intersection(set(wanted_label)):
            return label_name
        else:
            return "O"

    df[label_name] = list(df["labels"].apply(label_data))

    sentences = df.groupby("Sentence #")["text"].apply(list).values
    tags = df.groupby("Sentence #")[label_name].apply(list).values
    return sentences, tags

In [ ]:

from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from transformers import RobertaTokenizer
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


In [ ]:
self = classmethod("")


In [ ]:
class NER_Dataset_for_Adapter(Dataset):
    def __init__(self, tokenizer, df, label_name):
        self.label_name = label_name
        self.mode = "train"
        # 大數據你會需要用 iterator=True
        self.sentences, self.tags = get_training_data_by_df_according_to_label_name(df, label_name)
        self.len = len(self.sentences)


        labels = ["O", label_name]

        if self.mode != "test":
            labels = ["O", label_name]
            self.label_map = {}
            for i, label in enumerate(labels):
                self.label_map[label] = i

            possible_labels = np.array(range(len(labels))).reshape(-1, 1)
            self.oneHotEncoder = OneHotEncoder()
            self.oneHotEncoder.fit(possible_labels)
        else:
            self.label_map = None

        self.tokenizer = tokenizer  # RoBERTa tokenizer
        self.O_label = self.label_map["O"]

    def __getitem__(self, idx):
        if self.mode == "test":
            label_tensor = None
        else:
            label = ["O"] + self.tags[idx] + ["O"]

            label = np.array(label)
            label = label.reshape(-1,1)

            label = np.apply_along_axis(self.split_one_hot_multiTags, 1, label)
            label_tensor = torch.tensor(label, dtype = torch.float32)

        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = [tokenizer.cls_token]
        word_pieces += self.sentences[idx]
        word_pieces += [tokenizer.sep_token]

        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        # 將第一句包含 [SEP] 的 token 位置設為 0
        segments_tensor = torch.zeros_like(tokens_tensor)

        return (tokens_tensor, segments_tensor, label_tensor)

    def __len__(self):
        return self.len

    def split_one_hot_multiTags(self, tags):
        # tags = ['B-org|Party|String']
        tags = tags[0]
        tags = tags.split("|")


        tags_num = list(map(lambda x: self.label_map[x], tags))
        #[5, 20, 23]

        tags_num = np.array(tags_num).reshape(-1,1)

        tags_one_hot = self.oneHotEncoder.transform(tags_num).toarray()

        tags_one_hot = tags_one_hot.sum(axis = 0)

        #return torch.tensor(tags_one_hot, dtype = torch.float32)

        return tags_one_hot

In [ ]:
NER_multilabel_data_v2.csv

In [ ]:
tags

In [ ]:
self

In [ ]:
wanted_label